In [ ]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [31]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-04-28 22:07:31--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-04-28 22:07:31 (49.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [32]:
import requests
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [28]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [26]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   36G   65G  36% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   36G   65G  36% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    2.0G  1.1G  858M  56% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  3.6M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

In [22]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:

messages = [
    {"role": "user", "content": "I just discovered the course. Can I still join ?"},

]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
   # "temperature": 0.0,
    "do_sample": False,

}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])



Device set to use cuda


 I'm sorry, but I can't provide real-time or current information about specific courses or enrollment statuses. However, I can guide you on how to find out if you can still join a course. Here's what you can do:

1. Visit the course website: Go to the official website of the course you're interested in. Look for sections like "Enrollment," "Registration," or "Join Now."

2. Check the course status: Some courses have a status indicator that shows whether the course is open for enrollment, full, or closed. If the course is open, you can proceed to the next step.

3. Look for enrollment deadlines: If the course is open, check the enrollment deadline. Make sure you enroll before the deadline to secure your spot.

4. Contact the course instructor or admissions team: If you're unsure about the enrollment status or have any questions, reach out to the course instructor or the admissions team via email or phone. They can provide you with the most accurate information.

5. Check for alternative

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    messages = [
      {"role": "user", "content": "I just discovered the course. Can I still join ?"},
    ]

    generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
   # "temperature": 0.0,
    "do_sample": False,

    }
    output = pipe(messages, **generation_args)
    return output[0]['generated_text'].strip()

In [33]:
rag("I just discovered the course. Can I still join it?")

'I\'m sorry, but I can\'t provide real-time or current information about specific courses or enrollment statuses. However, I can guide you on how to find out if you can still join a course. Here\'s what you can do:\n\n1. Visit the course website: Go to the official website of the course you\'re interested in. Look for sections like "Enrollment," "Registration," or "Join Now."\n\n2. Check the course status: Some courses have a status indicator that shows whether the course is open for enrollment, full, or closed. If the course is open, you can proceed to the next step.\n\n3. Look for enrollment deadlines: If the course is open, check the enrollment deadline. Make sure you enroll before the deadline to secure your spot.\n\n4. Contact the course instructor or admissions team: If you\'re unsure about the enrollment status or have any questions, reach out to the course instructor or the admissions team via email or phone. They can provide you with the most accurate information.\n\n5. Check 